PS Надеюсь, полчаса от полуночи еще входят в 2 дня после дедлайна (я перепутала часовые пояса)

Для разметки корпуса я выбрала данный тегсет (https://universaldependencies.org/u/pos/index.html), поскольку он, на мой взгляд, достаточно универсален и имеет более "типичные" и обобщенные теги. И эти же теги используются в библиотеке Natasha.
В отобранных мной предложениях встречаются следующие сложные для pos-теггинга моменты: аббревиатуры, сокращения, омоформы, числительные (в том числе порядковые, такие как "13-й"), имена собственные и заимствованные слова. 

In [48]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()
from pymystem3 import Mystem
m = Mystem()
import re
import spacy
import pandas as pd

In [19]:
with open ('text.txt', encoding = 'utf-8') as file:
        text = file.read()

In [50]:
pip install openpyxl

     |████████████████████████████████| 242 kB 2.9 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [84]:
df = pd.read_excel("CORPUS.xlsx")
words_corpus = {}
for row in df.itertuples():
    words_corpus[str(row.word).lower()] = row.POS  

## Прогоняем текст через Mystem

In [20]:
ana = m.analyze(text)

In [90]:
words_mystem = {}
for word in ana:
    if 'analysis' in word:
        if word['analysis'] == []:
            words_mystem[word['text']] = 'X'
        else:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            if pos == 'A':
                pos = 'ADJ'
            elif pos == 'PR':
                pos = 'ADP'
            elif pos == 'V':
                pos = 'VERB'
            elif pos == 'CONJ':
                pos = 'CCONJ'
            elif pos == 'ADVPRO':
                pos = 'ADV'
            elif pos in ['APRO', 'SPRO']:
                pos = 'PRON'
            elif pos == 'ANUM':
                pos = 'NUM'
            elif pos == 'S':
                pos = 'NOUN'
            words_mystem[word['text'].lower()] = pos

## Прогоняем текст через pymorphy

In [93]:
text1 = re.sub(r'[^\w\s]','', text)
words_pymorphy = {}
for token in text1.split():
    ana2 = morph.parse(token)
    first = ana2[0]
    pos = str(first.tag.POS)
    if pos in ['ADJF', 'ADJS']:
        pos = 'ADJ'
    elif pos == 'PREP':
        pos = 'ADP'
    elif pos == 'CONJ':
        pos = 'CCONJ'
    elif pos == 'ADVB':
        pos = 'ADV'
    elif pos in ['PRTS', 'PRTF', 'GRND']:
        pos = 'VERB'
    elif pos == 'PRCL':
        pos = 'PART'
    elif pos == 'COMP':
        pos = 'ADJ'
    elif pos == 'INFN':
        pos = 'VERB'
    elif pos == 'NUMR':
        pos = 'NUM'
    elif pos == 'NPRO':
        pos = 'PRON' 
    elif pos == 'PRED':
        pos = 'ADV'
    elif pos == 'None':
        pos = 'X'
    words_pymorphy[str(first.word)] = pos

## Прогоняем текст через Natasha

In [34]:
!pip install natasha

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [37]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    PER,
    Doc
)

In [38]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
doc = Doc(text)

In [77]:
doc.segment(segmenter)
doc.tag_morph(morph_tagger)

In [95]:
words_natasha = {}
for token in doc.tokens:
    if token.pos != 'PUNCT':
        words_natasha[token.text.lower()] = token.pos

### Проверяем точность автоматического посттеггинга

In [82]:
from sklearn.metrics import accuracy_score

In [100]:
def accuracy (words_auto, words_corpus):
    gold = []
    results = []
    for key in words_corpus:
        gold.append(words_corpus[key])
        if key in words_auto:
            results.append(words_auto[key])
        else:
            results.append('None')  
    print("Accuracy: %.4f" % accuracy_score(results, gold)) 

In [101]:
#natasha
accuracy(words_natasha, words_corpus)

Accuracy: 0.8491


In [102]:
#pymorphy
accuracy(words_pymorphy, words_corpus)

Accuracy: 0.7862


In [103]:
#mystem
accuracy(words_mystem, words_corpus)

Accuracy: 0.8113


Лучший результат показала Natasha

In [ ]:
for i, s in enumerate(doc.sents):
    for t in s:

      if adj == '':
        if t.pos_ == 'ADJ':
          adj = t.text

      else :
        if t.pos_ == 'NOUN':
          print(adj, ' ', t.text)
          adj = ''
        elif t.pos_ != 'ADJ':
            adj = ''  
        else:
            adj = t.text

## Chunker

In [117]:
#adj + noun/ adp + noun/ pos + и + pos
def chunker (first, second, third, words_natasha):
    n_gramms = []
    words = ''
    if third == '':
        for token in doc.tokens:
            if words == '':
                if token.pos == first:
                    words = token.text.lower()
            else:
                if token.pos == second:
                    n_gramms.append(words+' '+token.text.lower())
                    words = ''
                elif token.pos != first:
                    words = ''
                else:
                    words = token.text.lower()
    else:
        words_3 = []
        for token in doc.tokens:
            if words_3 == []:
                if token.pos == first:
                    words_3.append(token.text.lower())
            elif len(words_3) == 1:
                if token.text.lower() == second:
                    words_3.append(token.text.lower())
                elif token.pos != first:
                    words_3 = []
                else:
                    words_3.append(token.text.lower()) 
            else:
                if token.pos == third:
                    words_3.append(token.text.lower())
                    n_gramms.append(words_3[0]+' '+words_3[1]+' '+words_3[2])    
                    words_3 = []
                elif token.text.lower() != second:
                    words_3 = []
                else:
                    words_3.append(token.text.lower())
    return n_gramms

In [126]:
print(chunker ('ADJ', 'и', 'ADJ', doc.tokens))

['железными и бетонными']


In [127]:
print(chunker ('VERB', 'и', 'VERB', doc.tokens))

['смотрят и веселятся']


In [128]:
print(chunker ('ADP', 'NOUN', '', doc.tokens))

['в пресс-службе', 'в ответ', 'на запрос', 'на вопрос', 'в мире', 'против кори', 'на платформе', 'на льду', 'о ступеньку', 'на стол', 'в печи']


In [129]:
print(chunker ('ADJ', 'NOUN', '', doc.tokens))

['бетонными опорами', 'стеклянных гирляндах', 'провисают провода', 'российские загсы', 'государственной регистрации', 'отечественной ономастики', '13-м знаком', '2011 году', 'зодиакальный круг', 'японские астрономы', '2002 г', 'абсолютный оффтопп', 'прямое отношение', 'замечательные кадры']


## 5-ое задание

Поскольку в предыдущей домашней работе я скачивала отзывы от кофе, было бы здорово добавить шаблоны "средний + NOUN" (потому что "средняя кислотность" - это скорее хорошо, а "средний вкус" - плохо), а также рассмотрела бы шаблоны "очень +  VERB", "очень + ADJ"